In [ ]:
import pandas as pd
import numpy as np
import itertools
from scipy.stats import fisher_exact
import shelve
import pickle
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
import csv
from dfply import *
import itertools
import os
import logging
from sys import getsizeof
import sklearn
import time
from sklearn.metrics import roc_auc_score
from catboost import Pool, cv
import xgboost
import catboost
import scipy.stats as st

import importlib
from joblib import Parallel, delayed
from joblib import parallel_backend

import ipynb.fs.full.preprocessing1

import dask.dataframe as dd
import dask

In [ ]:
datafolder = '/home/hoyinchan/blue/Data/data2021raw/'
data_onset = pd.read_csv(datafolder+'KUMC'+'/raw_OLD_FORMAT/'+'AKI_ONSETS.csv')
data_onset = data_onset[data_onset['BCCOVID'] == 1]
data_onset = data_onset.drop('BCCOVID',axis=1)
data_onset['ADMIT_DATE'] = pd.to_datetime(data_onset['ADMIT_YEAR'].astype(str)+'-01-02 00:00:00.000')
data_onset= data_onset.drop('ADMIT_YEAR',axis=1)
data_onset['NONAKI_ANCHOR'] = data_onset['ADMIT_DATE']+pd.to_timedelta(data_onset['NONAKI_SINCE_ADMIT'], "d") 
data_onset['AKI1_ONSET'] = data_onset['ADMIT_DATE']+pd.to_timedelta(data_onset['AKI1_SINCE_ADMIT'], "d") 
data_onset['AKI2_ONSET'] = data_onset['ADMIT_DATE']+pd.to_timedelta(data_onset['AKI2_SINCE_ADMIT'], "d") 
data_onset['AKI3_ONSET'] = data_onset['ADMIT_DATE']+pd.to_timedelta(data_onset['AKI3_SINCE_ADMIT'], "d") 
data_onset.to_parquet(datafolder+'KUMC'+'/raw/'+'AKI_ONSETS.parquet')
data_onset.to_csv(datafolder+'KUMC'+'/raw/'+'AKI_ONSETS.csv')

In [ ]:
def change_enc_raw(data2, colname):
    data=data2.copy()
    data.columns = colname
    data['ENCOUNTERID'] = data['ONSETS_ENCOUNTERID']
    data = data.drop('ONSETS_ENCOUNTERID',axis=1)
    return data

In [ ]:
def modify_dates(data2, feature_names, mode='int'):
    data=data2.copy()
    data_onset = pd.read_csv(datafolder+'KUMC'+'/raw/'+'AKI_ONSETS.csv')
    data = data.merge(data_onset[['PATID','ENCOUNTERID','ADMIT_DATE']].drop_duplicates(), on=['PATID','ENCOUNTERID'], how='left')
    for f in feature_names:
        time_feature = f+'_TIME'
        datetime_feature = f+'_DATE_TIME'        
        if mode != 'int':
            data['DAYS_SINCE_ADMIT'] = data['DAYS_SINCE_ADMIT'].map(np.floor).astype(int)        
        data[datetime_feature] = pd.to_datetime(data['ADMIT_DATE'])+pd.to_timedelta(data['DAYS_SINCE_ADMIT'], "d")+pd.to_timedelta(data[time_feature]+':00')
    return data

In [ ]:
def modify_dates2(data2, out_feature_names, mode='int'):
    data=data2.copy()
    data_onset = pd.read_csv(datafolder+'KUMC'+'/raw/'+'AKI_ONSETS.csv')
    data = data.merge(data_onset[['PATID','ENCOUNTERID','ADMIT_DATE']].drop_duplicates(), on=['PATID','ENCOUNTERID'], how='left')
    if mode != 'int':
        data['DAYS_SINCE_ADMIT'] = data['DAYS_SINCE_ADMIT'].map(np.floor).astype(int) 
    data[out_feature_names] = pd.to_datetime(data['ADMIT_DATE'])+pd.to_timedelta(data['DAYS_SINCE_ADMIT'], "d")
    return data

In [ ]:
def missolenous(data2):
    data=data2.copy()
#    data = data.drop(['ADMIT_DATE', 'MEDADMINID', 'MEDADMIN_PROVIDERID', 'MEDADMIN_DOSE_ADMIN_UNIT', 'PRESCRIBINGID'],axis=1)
#    data = data.drop(['BIRTH_TIME', 'BIRTH_TIME', 'GENDER_IDENTITY', 'PAT_PREF_LANGUAGE_SPOKEN', 'BIOBANK_FLAG', 'SEXUAL_ORIENTATION', ],axis=1)
#    data = data.drop(['PROVIDERID', 'DIAGNOSISID', 'ENC_TYPE', 'DX_POA'],axis=1, errors='ignore')
#    data = data.drop(['ADMIT_DATE', 'PPX', 'ENC_TYPE', 'PROCEDURESID'],axis=1, errors='ignore')
#    data = data.drop(['ADMIT_DATE', 'VITALID', 'MEASURE_TIME', 'VITAL_SOURCE', 'BP_POSITION'],axis=1, errors='ignore')
    data = data.drop(['ADMIT_DATE', 'PRESCRIBINGID', 'RX_PROVIDERID', 'RX_ORDER_TIME', 'RX_DOSE_ORDERED', 'RX_DOSE_ORDERED_UNIT', 'RX_DOSE_FORM', 'RX_PRN_FLAG', 'RX_ROUTE', 'RX_SOURCE', 'RX_DISPENSE_AS_WRITTEN'],axis=1, errors='ignore')
    data = data[[x for x in data.columns if 'RAW' not in x]]
    return data

In [ ]:
def batch(data, filename, feature_names, mode='int'):
#    datac = dd.read_csv(datafolder+'KUMC'+'/raw_OLD_FORMAT/'+filename+'.csv', blocksize=188e6)      
#    colnames = datac.columns
    data = change_enc_raw(data, colnames)
    data = modify_dates2(data, feature_names, mode=mode)
#    data = lab_proc(data)    
    data = missolenous(data)
#        data.to_pickle(datafolder+'KUMC'+'/raw/'+filename+'.pkl')
    return data

In [ ]:
vital = ['AKI_VITAL', ['MEASURE'], 'int']
px    = ['AKI_PX', [], 'int']
lab   = ['AKI_LAB', ['SPECIMEN', 'RESULT'], 'int']
dx    = ['AKI_DX', [], 'int']
dmed  = ['AKI_DMED', [], 'int']
demo  = ['AKI_DEMO', [], 'int']
amed  = ['AKI_AMED', ['MEDADMIN_START', 'MEDADMIN_STOP'], 'float']

In [ ]:
import os
files_lab = os.listdir(datafolder+'KUMC'+'/raw_OLD_FORMAT/'+'AKI_AMED_SPLIT/')

In [ ]:
#files_lab = files_lab[0:2]

In [ ]:
def readcsvchunk(filename, datafolder):
    fname = datafolder+'KUMC'+'/raw_OLD_FORMAT/'+'AKI_AMED_SPLIT/'+filename
    return pd.read_csv(fname, header=None)
amed = Parallel(n_jobs=63)(delayed(readcsvchunk)(flab, datafolder) for flab in files_lab)

In [ ]:
amed = Parallel(n_jobs=63)(delayed(batch)(flab_df, 'AKI_AMED', ['MEDADMIN_START', 'MEDADMIN_STOP'], 'float') for flab_df in amed)
amed_con = pd.concat(amed)
amed_con.to_parquet(datafolder+'KUMC'+'/raw/'+'AKI_AMED'+'.csv')

In [ ]:
pmed = pd.read_csv(datafolder+'KUMC'+'/raw_OLD_FORMAT/'+'AKI_PMED'+'.csv', low_memory=False)      
pmed = change_enc_raw(pmed, pmed.columns)
pmed = modify_dates(pmed, ['RX_ORDER'], mode='int')
pmed = missolenous(pmed)
pmed['RX_START_DATE'] = pmed['RX_ORDER_DATE_TIME'].dt.date
pmed['RX_END_DATE'] = pmed['RX_ORDER_DATE_TIME'].dt.date
#vital.to_parquet(datafolder+'KUMC'+'/raw/'+'AKI_VITAL'+'.parquet')

In [ ]:
def convert_parquest_to_csv(dilename, datafolder):
    if 'parquet' in dilename:
        data = pd.read_parquet(datafolder+'KUMC'+'/raw/'+dilename)
        data.to_csv(datafolder+'KUMC'+'/raw/'+dilename.replace('parquet', 'csv'))

In [ ]:
import os
files_lab = os.listdir(datafolder+'KUMC'+'/raw/')

In [ ]:
Parallel(n_jobs=10)(delayed(convert_parquest_to_csv)(flab, datafolder) for flab in files_lab)

In [ ]:
pmed.to_csv(datafolder+'KUMC'+'/raw/'+'AKI_PMED'+'.csv')

In [ ]:
pmed